In [1]:
!pip install opencv-python

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
import cv2

# Splitting dataset (splitting paths leading to images)

## creating dataset with paths to spectograms

In [3]:
# specify path to spectogram images
PATH = r"C:\Users\Ilan\Documents\ITC\11.Final_Project\2. Dataset 2\Data\images_original"

In [4]:
genres = os.listdir(PATH)

In [5]:
# getting path of all file names in directories
files = {}
for genre in genres:
    files[genre] = glob.glob(fr"{PATH}\{genre}\*png")

In [6]:
# creating dataframe wit paths
df = pd.DataFrame(columns = ['path', 'rgb_data', 'label'])
for genre in genres:
    df_aux = pd.DataFrame(files[genre], columns = ['path'])
    df_aux['label'] = genre
    df = df.append(df_aux, ignore_index=True, sort=False)

# verifying number of rows
df['label'].value_counts()

reggae       100
country      100
pop          100
hiphop       100
classical    100
metal        100
blues        100
disco        100
rock         100
jazz          99
Name: label, dtype: int64

In [7]:
#getting rgb data for all images
for index in df.index:
    img = cv2.imread(df['path'].loc[index])
    df['rgb_data'].loc[index] = np.array(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), dtype=np.float)

In [8]:
df.head()

,path,rgb_data,label
0,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
1,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
2,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
3,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
4,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues


## splitting dataset

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['rgb_data']
y = df['label']

#splitting data and stratifying by y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modelling

In [11]:
shape_of_input = df['rgb_data'][0].shape
shape_of_input

(288, 432, 3)

In [12]:
model = keras.Sequential([
    layers.Rescaling(1 / 255, input_shape=shape_of_input),
    
    layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    
    layers.Dropout(rate=0.3),
    
    layers.Dense(len(genres), activation='softmax')
])

In [13]:
print(f'Model input shape: {model.input_shape}, Model output shape: {model.output_shape}')

Model input shape: (None, 288, 432, 3), Model output shape: (None, 10)


In [14]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 288, 432, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 286, 430, 8)       224       
_________________________________________________________________
batch_normalization (BatchNo (None, 286, 430, 8)       32        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 141, 213, 16)      1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 141, 213, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 70, 106, 16)       0

In [15]:
model.compile(loss = 'crossentropy', optimizer = 'adam', metrics = 'accuracy')
model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=5)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).